In [ ]:
from finbourne_sdk_utils.jupyter_tools import toggle_code

"""Luminesce Providers and Views

Attributes
----------

"""

toggle_code("Toggle Docstring")

In [ ]:
from lusid.extensions import (
    SyncApiClientFactory,
    ArgsConfigurationLoader,
    EnvironmentVariablesConfigurationLoader,
    SecretsFileConfigurationLoader
)
from lusidjam import RefreshingToken
import os
import lusid as lu
import lusid.models as lm
import pandas as pd
import lusid_drive
import lusid_drive.rest
import lusid.rest
from lusid_drive.rest import ApiException
from pprint import pprint
import json

pd.set_option('display.max_colwidth', None)

# Authenticate to SDK
# Run the Notebook in Jupyterhub for your LUSID domain and authenticate automatically
secrets_path = os.getenv("FBN_SECRETS_PATH")
# Run the Notebook locally using a secrets file (see https://support.lusid.com/docs/how-do-i-use-an-api-access-token-with-the-lusid-sdk)
if secrets_path is None:
    secrets_path = os.path.join(os.path.dirname(os.getcwd()), "secrets.json")

# Initiate an API Factory which is the client side object for interacting with LUSID APIs
config_loaders=[
    ArgsConfigurationLoader(access_token = RefreshingToken(), app_name = "LusidJupyterNotebook"),
    EnvironmentVariablesConfigurationLoader(),
    SecretsFileConfigurationLoader(secrets_path)]
api_factory = lusid.extensions.SyncApiClientFactory(config_loaders=config_loaders)
 
# Confirm success by printing SDK version
api_status = pd.DataFrame(api_factory.build(lu.ApplicationMetadataApi).get_lusid_versions().to_dict())
display(api_status)

In [3]:
# here's some sample equities we've prepared earlier
uk_equity_instruments = pd.read_csv('_data/uk_equity_instruments.csv')
us_equity_instruments = pd.read_csv('_data/us_equity_instruments.csv')

In [4]:
# don't show exception if error is due to upsert
def exception_guard(e, code):
    return e.status and e.status != '400 Bad Request' and e.body and json.loads(e.body)["code"] == code

scope = 'FBNUniversity' # str | The scope of the portfolio.

In [5]:

# create a request to upsert a bunch of instruments.
def generate_upsert_instrument_request(api_instance, name, ccy, figi, isin, other_identifier_name, other_identifier):
    request_id = name + isin
    request ={"name":name,
              "identifiers":{ 
                  "Figi":{"value":figi},
                  "Isin":{"value":isin},
                  other_identifier_name:{"value":other_identifier}
                },
              "definition":lm.Equity(
                              instrument_type = "Equity",
                              dom_ccy = ccy,
                              identifiers = {}
                          )
    }
    return request_id, request

In [6]:
# Create an instance of the API class
api_instance = api_factory.build(lu.InstrumentsApi)

# We're going to load some american and UK equities into LUSID

other_identifier_name = 'Sedol'
ccy = 'GBP'
upsert_body_tuple = (generate_upsert_instrument_request(api_instance, name, ccy, figi, isin, other_identifier_name, other_identifier)
               for 
               index, ticker, name, _, isin, other_identifier, figi
               in uk_equity_instruments.itertuples())
uk_equity_body_dict = {request_id:request_body for request_id,request_body in upsert_body_tuple}
other_identifier_name = 'Cusip'
ccy = 'USD'
upsert_body_tuple = (generate_upsert_instrument_request(api_instance, name, ccy, figi, isin, other_identifier_name, other_identifier)
               for 
               index, ticker, name, _, isin, other_identifier, figi
               in us_equity_instruments.itertuples())
us_equity_body_dict = {request_id:request_body for request_id,request_body in upsert_body_tuple}

request_body_dict = us_equity_body_dict | uk_equity_body_dict

try:
    # UpsertInstruments: Upsert instruments
    api_response = api_instance.upsert_instruments(request_body_dict, scope=scope)
#     pprint(api_response)
except lusid.rest.ApiException as e:
    print("Exception when calling InstrumentsApi->upsert_instruments: %s\n" % e)

In [7]:
api_instance = api_factory.build(lu.TransactionPortfoliosApi)
code = 'Module-T2-3' # str | The code of the portfolio. Together with the scope this uniquely identifies the portfolio.
create_transaction_portfolio_request = {"displayName":"Training module T2.3", "baseCurrency":"GBP", "code":code} # UpdatePortfolioRequest | The updated portfolio definition.
try:
    # CreatePortfolio: Create portfolio
    api_response = api_instance.create_portfolio(scope, create_transaction_portfolio_request)
except lusid.rest.ApiException as e:
    if not exception_guard(e, 112):
        print(e)
        

In [ ]:

from lusid_drive.extensions import (
    SyncApiClientFactory,
    ArgsConfigurationLoader,
    EnvironmentVariablesConfigurationLoader,
    SecretsFileConfigurationLoader
)
from lusid_drive.models import *

# Authenticate to SDK
# Run the Notebook in Jupyterhub for your LUSID domain and authenticate automatically
secrets_path = os.getenv("FBN_SECRETS_PATH")
api_url_drive = os.getenv("FBN_DRIVE_API_URL")
# Run the Notebook locally using a secrets file (see https://support.lusid.com/docs/how-do-i-use-an-api-access-token-with-the-lusid-sdk)
if secrets_path is None:
    secrets_path = os.path.join(os.path.dirname(os.getcwd()), "secrets.json")

# Initiate an API Factory which is the client side object for interacting with LUSID APIs
config_loaders_drive=[
    ArgsConfigurationLoader(access_token = RefreshingToken(), app_name = "LusidJupyterNotebook", api_url=api_url_drive),
    EnvironmentVariablesConfigurationLoader(),
    SecretsFileConfigurationLoader(secrets_path)]
api_factory_drive = lusid_drive.extensions.SyncApiClientFactory(config_loaders=config_loaders_drive)
 
folders_api = api_factory_drive.build(lusid_drive.api.FoldersApi)

# let's chuck a file containing a randomly selected subset of 25 instruments with their sectors into drive for some examples:
subset_file = open('_data/instrument_subset.csv').read()
create_folder = {"path":"/","name":"FBN-University"} # CreateFolder | A CreateFolder object that defines the name and path of the new folder

try:
    # [EARLY ACCESS] CreateFolder: Create a new folder in LUSID Drive
    api_response = folders_api.create_folder(create_folder)
    pprint(api_response)
except ApiException as e:
    if not exception_guard(e, 664):
        print("Exception when calling FilesApi->create_file: %s\n" % e)
    
files_api = api_factory_drive.build(lusid_drive.api.FilesApi)
    
x_lusid_drive_filename = 'instrument_subset_T02003.csv' # str | File name.
x_lusid_drive_path = '/FBN-University/' # str | File path.
content_length = len(subset_file.encode('UTF-8'))
body = subset_file # str | 

try:
    # [EARLY ACCESS] CreateFile: Uploads a file to Lusid Drive. If using an SDK, consider using the UploadAsStreamAsync function for larger files instead.
    api_response = files_api.create_file(x_lusid_drive_filename, x_lusid_drive_path, content_length, body)
except lusid_drive.rest.ApiException as e:
    if not exception_guard(e, 671):
        print("Exception when calling FilesApi->create_file: %s\n" % e)

display(api_response)

# Providers and Views


A provider is a component that enables you to write a Luminesce SQL query for a data source in situ, without first having to extract, transform or load data from that source.

We supply providers for numerous data sources, including for the investment data stored in LUSID itself and for files stored in Drive.

In this part of the course we will cover:

- Data providers
- Direct providers
- Write providers
- Provider parameters
- Using Luminesce views
- Writing Luminesce views

## Data providers

A data provider is designed to query a data source whose shape is known. It therefore returns a table of results with a fixed number of fields (columns).

Most of the providers we supply to query the investment management data stored in LUSID itself are data providers with a fixed number of fields, making it easier to query the data using standard SQL query syntax and luminesce extensions.

A data provider does not have to query a datasource, as long as it returns some data.For example, we supply providers that perform calculations on demand, such as `Lusid.Portfolio.Valuation`, where the data returned doesn't actually exist anywhere but rather is calculated each time the provider is queried.

## Direct providers 

A direct provider is designed to query a data source whose shape is not known, and thus cannot return a table of results with a fixed number of fields (columns).

A Luminesce query for a direct provider uses an arbitrary syntax, which may differ for each provider, and is defined by the writer of the provider. 

    @variable = use Some.Provider [wait]
    <arbitrary-syntax>
    enduse;
    select * from @variable


    
Some direct providers can also interpolate table and scalar parameters into their arbritrary syntax, enabling the use of parameters within the direct provider query using the WITH syntax:

    @@today = select strftime('%Y%m%d', 'now');
    @table_of_results = use Drive.Csv with @@today
    --file=/trade-files/eod-{@@today}.csv
    --noHeader
    --names=Equity,Units,Cost
    enduse;
    select * from @table_of_results


## Write providers

As mentioned earlier, data providers aren't limited to reading data from a datasource. Some providers' primary role is to write data to some target.

Using a write provider is straightforward. Query the Provider passing the data you'd like to write as parameters to fields specified by the provider, for example:

    @table_of_data = <select-statement>;
    select * from <Provider>.Writer where toWrite = @table_of_data;
    
By convention, many write providers accept a table of values as a parameter, and specify the expected columns in their fields.

## Examples of Lusid Providers

In this section we'll demonstrate some simple examples using the different types of provider.

In [ ]:
# Only required if running locally:
!pip3 install -U lumipy

In [10]:
## Only required if running locally:
import os
from IPython.core.magic import (register_line_cell_magic)
from lumipy.client import Client
from lusidjam import RefreshingToken

token = RefreshingToken()
lumi_url = os.getenv("FBN_LUMI_API_URL")

@register_line_cell_magic
def luminesce(line, cell=None):
    query = cell if cell is not None else line

    try:
        lm_client = Client(token=token, api_url=lumi_url)
    except TypeError:
        # Attempt to use V2 SDK syntax if V1 syntax fails.
        # This gives V2 SDK support for luminesce magic
        lm_client = Client(access_token=token, api_url=lumi_url)

    df = lm_client.query_and_fetch(query)
            
    return df

# In an interactive session, we need to delete to avoid name conflicts for automagic to work on line magics.
del luminesce

### Data Providers


#### Lusid.Instrument.Equity Provider

Luminesce has a number of `Lusid.Instrument` providers that enable you to write a Luminesce query to retrieve instruments mastered in LUSID.

The generic `Lusid.Instrument` provider retrieves basic information about instruments of any asset class. Dedicated providers retrieve instruments of a particular asset class, for example `Lusid.Instrument.Bond` retrieves the full economic definition of bond instruments.

We'll use the `Lusid.Instrument` Provider to query for the equities we have stored in Lusid. Then we'll use the `Lusid.Instrument.Equity` Provider to look at the full economic definition of stored equity instruments.

In [ ]:
%%luminesce
SELECT *
FROM Lusid.Instrument
LIMIT 100

The `Lusid.Instrument.Equity` Provider has a couple of extra fields: DomCcy & Identifiers, and will only return Instruments which are equities.

In [ ]:
%%luminesce
SELECT DisplayName, DomCcy
FROM Lusid.Instrument.Equity
LIMIT 100

#### Lusid.Portfolio Provider

The `Lusid.Portfolio` provider enables you to write a Luminesce SQL query that retrieves information about one or more portfolios.

In [ ]:
%%luminesce
SELECT *
FROM Lusid.Portfolio
LIMIT 100

Here we've returned all metadata about our Portfolios in Lusid.

#### Lusid.Tools Provider

The Lusid.Tools Provider does not query any data. It manipulates the input parameters to return a table. 

The `Tools.JsonExpand` provider enables you to write a Luminesce query that parses a JSON document into a table of constituent data objects, much like the SQLite json.tree function. We'll use this to demonstrate a data provider that does not query for any external data, instead manipulating the provided parameters.

In [ ]:
%%luminesce
@@jsn = SELECT '{
    "characters":[
        {"name":"marge",
         "age":36},
        {"name":"homer",
         "age":40},
        {"name":"bart",
         "age":9},
        {"name":"lisa",
         "age":12}
    ]
}';
SELECT * FROM Tools.JsonExpand WHERE JsonString = @@jsn


In [ ]:
%%luminesce
@@jsn = SELECT '{
    "characters":[
        {"name":"marge",
         "age":36},
        {"name":"homer",
         "age":40},
        {"name":"bart",
         "age":9},
        {"name":"lisa",
         "age":12}
    ]
}';
@@selection = select "$.characters[0]";
select * from Tools.JsonExpand where JsonString = @@jsn and RootPath = @@selection

### Direct Providers

#### Drive.Csv Provider

The Drive.Csv provider enables you to write a Luminesce query that extracts data from one or more CSV or similar pipe-delimited or row-based text files stored in Drive.
We can pass a number of options to the Drive CSV provider, using the syntax `--<option>[=<value>]`

In [ ]:
%%luminesce
@x = use Drive.Csv
--file=/finbourne university/instrument_subsetT2.3.csv
--noHeader
--names=Name,Sector,Identifier
enduse;
select * from @x

Here we've read all data from the `/finbourne university/instrument_subsetT2.3.csv` file in drive. The file has no headers, so we've specified the no headers option, and provided a set of column names to the Provider.

### Write Providers

#### Lusid.Instrument.Equity.Writer

Many of our Lusid write providers accept a `TableParameter` used to write multiple resources to some part of Lusid.


Now we'll use a Write Provider to create some equities by passing a table of equities as a parameter.

First let's take a look at the table parameter used by the `Lusid.Instruments.Equities` provider, and what the table it expects looks like.

In [ ]:
%%luminesce 
select FieldName, TableParamColumns from Sys.Field where TableName = 'Lusid.Instrument.Equity.Writer' AND DataType = 'Table';

We can see there's a `ToWrite` column which accepts a table of equities. By convention, many Lusid write providers use `toWrite` as a table parameter, returning the written values in the table, and some error information.

We recommend examining the results of every write query using one or more of the `WriteError`, `WriteErrorCode` and `WriteErrorDetail` fields.

For each record in the table of data to write, `Lusid.Portfolio.Writer` returns an error code. If the operation is successful, the error code is 0. If unsuccessful, a positive error code and explanation help you discover why LUSID considers the operation invalid.  

In [ ]:
%%luminesce
select * from Sys.Field where TableName = 'Lusid.Instrument.Equity.Writer';

We'll upsert some equities. Looking at the [knowledge-base](https://support.lusid.com/knowledgebase/article/KA-01685/en-us), we can see that the `Equity.Writer` Provider must have a DisplayName, at least one unique-identifier, and a `DomCcy`.

In [ ]:
%%luminesce
@inserted_values = SELECT 
                    'T2.3-Fake-Equity-1' as DisplayName,
                    '0000000-000001' as Figi, 
                    'GBP' as DomCcy
                   UNION
                   SELECT
                   'T2.3 Fake Equity-2' as DisplayName,
                   '0000000-000002' as Figi,
                   'CHF' as DomCcy;

SELECT *
FROM @inserted_values;

In [ ]:
%%luminesce
@inserted_values = SELECT 
                    'T2.3-Fake-Equity-1' as DisplayName,
                    '0000000-000001' as Figi, 
                    'GBP' as DomCcy
                   UNION
                   SELECT
                   'T2.3 Fake Equity-2' as DisplayName,
                   '0000000-000002' as Figi,
                   'CHF' as DomCcy;

SELECT * FROM Lusid.Instrument.Equity.Writer WHERE toWrite = @inserted_values;

As you can see, a table is returned with the values we've written, some values auto-created by LUSID, and error information (if there was an error).

#### Drive.SaveAs

The `Drive.SaveAs` Provider is a Direct Provider that allows you to write tables of data to different types of file. As its a direct provider that has its own arbritrary syntax used to tell the Provider what data to write, to which file type, and how to write the data.

In [ ]:
%%luminesce
@inserted_file = SELECT
                'Marge' as CharacterName,
                '35' as Age, 
                'Married' as Status;
SELECT * FROM @inserted_file;

In [ ]:
%%luminesce
@inserted_file = SELECT
                'Marge' as CharacterName,
                '35' as Age, 
                'Married' as Status;
                 
@x = use Drive.SaveAs with @inserted_file
--path=/finbourne university
--fileNames=simpsons
--type=Csv
enduse;
select * from @x

We can see from the table that a simpsons.csv file has been written to drive, containing one row.

## Luminesce views

You can use the Sys.Admin.SetupView direct provider supplied by FINBOURNE to create a ‘view’ that can access any number of other Luminesce providers (or existing views).

The goal is to enable end users to write simplified or more performant queries using this view instead of the underlying providers directly, obscuring complexity, increasing productivity and promoting adoption.

Provider creation syntax

    @x = use Sys.Admin.SetupView [with @@scalar_var1 [, @@scalar_var2...]]
    <options-section>
    ----
    <sql-section>
    enduse;
    -- The following statement is optional but useful while creating the view to test it returns meaningful data
    select * from @x

SetupView works similarly to other direct Providers. You create a variable using `SetupView`, and can pass parameters using the with syntax.
You can specify options on the View Provider as usual, by passing `--<key>=<value>` pairs to the Provider.
Once you have configured the provider, use 4 dashes `----` to separate options from the SQLite operation you'd like to use in your View.

In [ ]:
%%luminesce
@x = use Sys.Admin.SetupView
--provider=Views.FBNUNIVIEW
----
SELECT #SELECT {
  { EquityName: i.DisplayName },
}
FROM Lusid.Instrument.Equity i
WHERE i.DisplayName LIKE 'A%'
enduse;

Here we've created a view called `Views.FBNUNIVIEW` which selects any DisplayNames that begin with 'A' from `Lusid.Instrument.Equity` and maps the `DisplayName` column to `EquityName`.

The `#SELECT` is special syntax used to signify to luminesce that this query should be optimized, described in the [setupview article](https://support.lusid.com/knowledgebase/article/KA-01767/en-us) in the knowledgebase.

We can now use this view to query data:

In [ ]:
%%luminesce
SELECT * FROM Views.FBNUNIVIEW 
LIMIT 10;

We've selected 10 of the Equities that would've been returned from our view.